# Regression

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost
import math
#from __future__ import division
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import  tree, linear_model
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plot
from sklearn.metrics import mean_squared_error, r2_score
import datetime as dt
import pickle

# Merging excel

In [15]:
df=pd.read_csv('C:/Users/v-savrav/Desktop/30/InvociePrediction30days_02Oct2019_v1.csv')

In [16]:
df.drop(['ReceiptDate', 'FirstPOCAssignmentDate', 'LastBusinessUserDate',
        'Doc. Date',
       'Clrng doc.', 'Pmnt date', 'Reference',
       'Clearing', 'Pstng Date', 'CoCd', 'Net due dt', 'Year',
       'Year/month', 'Period', 
       'Bline Date', 'Reference Key', 'Entry Date', 'SubmitDate'
], axis=1, inplace=True)

In [17]:
df.columns

Index(['Country', 'Region', 'Frequency', 'CC', 'received_days',
       'processing_days', 'days_to_approval', 'days_to_be_cleared', 'PaidDays',
       'NewPaymentTerm', 'DocumentNo', 'Type', 'Amount in doc. curr.', 'Curr.',
       'Vendor', 'Discount base amount', 'Discount amount'],
      dtype='object')

In [18]:
#X=df[['CC','received_days','processing_days','days_to_approval','days_to_be_cleared',
 #      'Vendor',  'NewPaymentTerm', 'Amount in doc. curr.',
  #     'Curr.', 'Type',  'Discount amount',
   #    'Discount base amount', 'Country', 'Region', 'Frequency']]

#Modified by Zaheer
X=df[['CC','received_days','processing_days','days_to_approval','days_to_be_cleared'
      ,'Vendor',
       'NewPaymentTerm', 'Amount in doc. curr.','Curr.','Type',  
            'Discount amount','DocumentNo', 'Discount base amount','Country',
       'Region', 'Frequency']]

# Preprocessing

In [19]:
from sklearn import  preprocessing # used for label encoding and imputing NaNs

# One-hot encode the data using pandas get_dummies
X.fillna(0,inplace=True)
for c in X.columns:
    if X[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X[c].values)) 
        X[c] = lbl.transform(list(X[c].values))

C:\Users\v-savrav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [20]:
X_train, X_test = train_test_split(X, test_size = 0.25, random_state = 42)

In [29]:
len(X_test)

56245

In [22]:
# Split the data into training and testing sets
def model_prediction(X_train, X_test,y_train, y_test,model_name):
    
    xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                               colsample_bytree=1, max_depth=7)
    params = {'n_estimators': 500, 'max_depth': 6, 'min_samples_split': 4,
          'learning_rate': 0.05, 'loss': 'ls'}
    clf = ensemble.GradientBoostingRegressor(**params)
    xgb.fit(X_train,y_train)
    clf.fit(X_train,y_train)
    filename = 'C:/Users/v-savrav/Desktop/30/models/' + model_name+'_model.sav'
    pickle.dump(clf, open(filename, 'wb'))
    filename1 ='C:/Users/v-savrav/Desktop/30/models/' + model_name+'_model1.sav'
    pickle.dump(xgb, open(filename1, 'wb'))
    pred1 = xgb.predict(X_test)
    pred2 = clf.predict(X_test)
    finalpred =(pred1+pred2)/2
    mae = mean_absolute_error(finalpred ,y_test)
    return finalpred,y_test,mae


# Paid days Prediction

In [23]:
#df['var_paymenent_days'] = df['PaidDays'] - df['NewPaymentTerm'] 
target=df['PaidDays']
y_train, y_test = train_test_split(target, test_size = 0.25, random_state = 42)

In [24]:
paid_days_prediction,actual_paid,mae= model_prediction(X_train, X_test,y_train, y_test,'stage4_paid')

C:\Users\v-savrav\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:43:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [28]:
mae

1.4901482545323999

In [19]:
df['PaidDays'].value_counts()

 26     47427
 29     37735
 28     26195
 27     25611
 25      9180
 24      8533
 22      5655
 23      4825
 21      4180
 30      2858
 20      2513
 12      2434
 14      2371
 1       2305
 10      2130
 7       2102
 15      2020
 19      2017
 13      1933
 18      1871
 11      1870
 6       1823
 2       1712
 31      1659
 8       1570
 3       1563
 17      1535
 9       1496
 4       1477
 32      1439
        ...  
 57       241
 54       224
 53       206
 52       205
 51       189
 58       150
 60       144
 59       137
 0        125
-1          6
-2          4
-4          4
-6          2
-29         2
-11         1
-3          1
-5          1
-7          1
-8          1
-10         1
-108        1
-13         1
-238        1
-16         1
-26         1
-30         1
-35         1
-65         1
-106        1
-142        1
Name: PaidDays, Length: 82, dtype: int64

In [14]:
X_test.to_excel('C:/Users/v-savrav/Desktop/30/Data_test/Stage4.xlsx')

In [30]:
paiddf = pd.DataFrame({'Paid - Act':actual_paid, 'Paid - Pr':paid_days_prediction})


In [31]:
final=pd.concat([paiddf, X_test],axis=1)
final['stage']='stage4'


In [34]:
final.to_excel('C:\\Users\\v-savrav\\Desktop\\30\\Train_results\\stage4_results.xlsx')

In [ ]:
final.columns #Added by Zaheer

In [ ]:
#Added by Zaheer
ff=final.rename(columns={'Amount in doc. curr.':'Amount',
             'NewPaymentTerm':'Pay Term'})

In [ ]:
#Added by Zaheer and removed columns (,'Rcvd -Act','Rcvd  -Pr','Procesing - Act',
##'Procesing  - Pr''Approval - Act','Approval - Pr',)
final = ff[['DocumentNo','Paid - Act','Paid - Pr','CC','Vendor',
'Pay Term','Amount','Curr.','Type','Discount amount','Country','Region','Frequency']]

In [ ]:
#Added by Zaheer
final['stage']='stage4'

In [ ]:
final=final.dropna()

In [ ]:
stage4=final

In [ ]:
stage4

In [ ]:
stage4.to_excel('C:/Users/v-shzah/Documents/Ganesh Data/Train - All Payment Terms Output Files/stage4_Train.xlsx')

In [ ]:
import pyodbc 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=azgovprdsql11;'
                      'Database=PWT;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [ ]:
for index,row in stage4.iterrows():
    cursor.execute('''INSERT INTO [All_Payment_Term_Train_Stage4_Output]
                   (
                  [DocumentNo],
                  [Paid - Act],[Paid - Pr],
                  [CC],[Vendor],[Pay Term],[Amount],[Curr.],[Type],
                  [Discount amount],[Country],[Region],[Frequency],[stage] )
                   
    values (?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?)''',
    row['DocumentNo'],row['Paid - Act'],row['Paid - Pr'],
                      row['CC'],row['Vendor'],row['Pay Term'],row['Amount'],row['Curr.'],
                      row['Type'],row['Discount amount'],row['Country'],row['Region'],
                      row['Frequency'],row['stage'] )

In [ ]:
conn.commit()
conn.close()